In [1]:
# import libraries
library(dplyr)
library(caret)
library(scales)

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Warning message:
"package 'lattice' was built under R version 3.6.3"Loading required package: ggplot2
Warning message:
"package 'scales' was built under R version 3.6.3"

In [2]:
# read data
data = read.csv("prlmis-data-full.csv", header=TRUE, fileEncoding="UTF-8-BOM")

# remove alternate response variables
data = select(data, -c("PRLANY","PRLMISAB"))

# update categorical values from raw dataset to be descriptive
data$YEAR <- replace(data$YEAR, data$YEAR=="15", "2015")
data$YEAR <- replace(data$YEAR, data$YEAR=="16", "2016")
data$YEAR <- replace(data$YEAR, data$YEAR=="17", "2017")

data$AGECAT <- replace(data$AGECAT, data$AGECAT=="1", "12-17")
data$AGECAT <- replace(data$AGECAT, data$AGECAT=="2", "18-25")
data$AGECAT <- replace(data$AGECAT, data$AGECAT=="3", "26-35")
data$AGECAT <- replace(data$AGECAT, data$AGECAT=="4", "36-49")
data$AGECAT <- replace(data$AGECAT, data$AGECAT=="5", "50+")

data$SEX <- replace(data$SEX, data$SEX=="0", "Male")
data$SEX <- replace(data$SEX, data$SEX=="1", "Female")

data$MARRIED <- replace(data$MARRIED, data$MARRIED=="0", "Unmarried")
data$MARRIED <- replace(data$MARRIED, data$MARRIED=="1", "Divorced")
data$MARRIED <- replace(data$MARRIED, data$MARRIED=="2", "Widowed")
data$MARRIED <- replace(data$MARRIED, data$MARRIED=="3", "Married")
data$MARRIED <- replace(data$MARRIED, data$MARRIED=="4", "Married")

data$EDUCAT <- replace(data$EDUCAT, data$EDUCAT=="1", "School Age")
data$EDUCAT <- replace(data$EDUCAT, data$EDUCAT=="2", "Some HS")
data$EDUCAT <- replace(data$EDUCAT, data$EDUCAT=="3", "HS grad")
data$EDUCAT <- replace(data$EDUCAT, data$EDUCAT=="4", "Some College")
data$EDUCAT <- replace(data$EDUCAT, data$EDUCAT=="5", "College Grad")

data$EMPLOY18 <- replace(data$EMPLOY18, data$EMPLOY18=="0", "Unemployed")
data$EMPLOY18 <- replace(data$EMPLOY18, data$EMPLOY18=="1", "Part-Time")
data$EMPLOY18 <- replace(data$EMPLOY18, data$EMPLOY18=="2", "Full-Time")

# wrong
data$CTYMETRO <- replace(data$CTYMETRO, data$CTYMETRO=="1", "Rural")
data$CTYMETRO <- replace(data$CTYMETRO, data$CTYMETRO=="2", "Small")
data$CTYMETRO <- replace(data$CTYMETRO, data$CTYMETRO=="3", "Large")
data$CTYMETRO <- replace(data$CTYMETRO, data$CTYMETRO=="0", "na")

# convert categorical variables to factors
data$YEAR<-as.factor(data$YEAR)
data$AGECAT<-as.factor(data$AGECAT)
data$SEX<-as.factor(data$SEX)
data$MARRIED<-as.factor(data$MARRIED)
data$EDUCAT<-as.factor(data$EDUCAT)
data$EMPLOY18<-as.factor(data$EMPLOY18)
data$CTYMETRO<-as.factor(data$CTYMETRO)
data$PRLMISEVR<-as.factor(data$PRLMISEVR)
data$HEROINEVR<-as.factor(data$HEROINEVR)

In [3]:
# train/test split
set.seed(123)

fractrain = 0.75
fractest = 1-fractrain
flag <- sort(sample(dim(data)[1], dim(data)[1]*fractest, replace = FALSE))
train <- data[-flag,]
test <- data[flag,]
dim(train)
dim(test)

[1] 127738     19

[1] 42579    19

In [4]:
# build logistic regression model to evaluate feature importance
glm1 <- glm(PRLMISEVR ~ ., data = train, family="binomial")

# get confidence interval of model coefficients
coeffs <- confint(glm1, level=0.9)

# order by most important coefficients
coeffs.sorted <- coeffs[order(abs(coeffs[,1]), decreasing=TRUE),]

# calculate change in odds of drug misuse by taking exponent of coefficients
agecat50.odds <- round(exp(coeffs.sorted[1,]), 2)
unmarried.odds <- round(exp(coeffs.sorted[5,]), 2)
halucng.odds <- round(exp(coeffs.sorted[6,]), 2)
amphetmn.odds <- round(exp(coeffs.sorted[7,]), 2)
heroinevr.odds <- round(exp(coeffs.sorted[8,]), 2)

In [5]:
summary(glm1)


Call:
glm(formula = PRLMISEVR ~ ., family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.2751  -0.3853  -0.3213  -0.2654   2.7901  

Coefficients: (2 not defined because of singularities)
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.359326   0.089576 -26.339  < 2e-16 ***
YEAR2016           -0.637541   0.067900  -9.389  < 2e-16 ***
YEAR2017           -0.653786   0.068159  -9.592  < 2e-16 ***
AGECAT18-25        -0.068249   0.049394  -1.382 0.167049    
AGECAT26-35         0.101249   0.049802   2.033 0.042050 *  
AGECAT36-49        -0.252396   0.051236  -4.926 8.39e-07 ***
AGECAT50+          -0.663550   0.053642 -12.370  < 2e-16 ***
SEXMale             0.115354   0.021588   5.344 9.12e-08 ***
MARRIEDMarried      0.113076   0.032855   3.442 0.000578 ***
MARRIEDUnmarried   -0.541652   0.065245  -8.302  < 2e-16 ***
MARRIEDWidowed     -0.012463   0.049024  -0.254 0.799322    
EDUCATHS grad      -0.11296

In [6]:
# get confidence interval of model coefficients
coeffs <- confint(glm1, level=0.9)
coeffs.exp <- exp(coeffs)
coeffs.exp

Waiting for profiling to be done...


,5 %,95 %
(Intercept),0.08155898,0.1095129
YEAR2016,0.47230410,0.5905393
YEAR2017,0.46449706,0.5812739
AGECAT18-25,0.86114148,1.0130832
AGECAT26-35,1.01951492,1.2010142
AGECAT36-49,0.71413192,0.8452416
AGECAT50+,0.47148222,0.5624784
SEXMale,1.08311850,1.1628365
MARRIEDMarried,1.06079151,1.1818724
MARRIEDUnmarried,0.52207919,0.6471010


In [8]:
# order by most important coefficients
coeffs.sorted <- coeffs[order(abs(coeffs[,1]), decreasing=TRUE),]
coeffs.sorted

,5 %,95 %
(Intercept),-2.506428851,-2.21171282
YEAR2017,-0.766800057,-0.54253321
AGECAT50+,-0.751873893,-0.57540250
YEAR2016,-0.750132217,-0.52671907
MARRIEDUnmarried,-0.649935989,-0.43525292
HALUCNG,0.459178540,0.49428094
AMPHETMN,0.388565599,0.44530234
HEROINEVR1,0.370454732,0.70510903
AGECAT36-49,-0.336687575,-0.16813276
TRQLZRS,0.335153048,0.37328038


In [9]:
# calculate change in odds of drug misuse
agecat50.odds <- round(exp(coeffs.sorted[1,]), 2)
unmarried.odds <- round(exp(coeffs.sorted[5,]), 2)
halucng.odds <- round(exp(coeffs.sorted[6,]), 2)
amphetmn.odds <- round(exp(coeffs.sorted[7,]), 2)
heroinevr.odds <- round(exp(coeffs.sorted[8,]), 2)

In [11]:
# output expected impact on odds of drug misuse
df <- data.frame(Factor=c("Age 50+", "Unmarried", "Halucinogen Use", "Amphetamin Use", "Ever used Heroin"),
                 Min=c(scales::percent(-(1-agecat50.odds[[2]])),
                      scales::percent(-(1-unmarried.odds[[2]])),
                      scales::percent(halucng.odds[[1]]-1),
                      scales::percent(amphetmn.odds[[1]]-1),
                      scales::percent(heroinevr.odds[[1]]-1)
                     ),
                 Max=c(scales::percent(-(1-agecat50.odds[[1]])),
                      scales::percent(-(1-unmarried.odds[[1]])),
                      scales::percent(halucng.odds[[2]]-1),
                      scales::percent(amphetmn.odds[[2]]-1),
                      scales::percent(heroinevr.odds[[2]]-1)
                     )
                 )
df

Factor,Min,Max
Age 50+,-89%,-92%
Unmarried,-35%,-48%
Halucinogen Use,58%,64%
Amphetamin Use,47%,56%
Ever used Heroin,45%,102%
